In [ ]:
library(dplyr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [ ]:
df <- read.csv("305_Student_Data.csv")

### Data Analysis
Among the students in 2023 Fall session of STAT404, 21 completed a survey about their performance in the pre-requisite for the course, STAT305. gasoline consumption was measured in the 50 states and the District of Columbia in the United States by the Federal Highway Administration.
The response $y$ is the final STAT305 grade of the students.
Four predictor variables are included:

> $x_1$: Weekly study time outside of class (hours);

> $x_2$: Number of Class absences;

> $x_3$: Session of Enrollment in the Course;

> $x_4$: Course Enjoyment (Ranked 1-5, 1 being awful, 5 being amazing);

Notationally, $k=4$ and $N=21$.

In [ ]:
df <- df %>%
    mutate(Enrollment.time = recode(Enrollment.time, '2023 summer' = '2023 Summer'))

In [ ]:
head(df)

,Overall.grade,Weekly.study.hours,Class.absences,Enrollment.time,Enjoyment
,<int>,<int>,<int>,<chr>,<int>
1,93,5,1,2021 Fall,5
2,95,17,0,2019 Winter,5
3,100,3,0,2023 Summer,5
4,95,7,1,2022 Fall,3
5,91,7,0,2022 Fall,3
6,85,8,8,2023 Summer,5


## Fitting linear models

Fitting a linear model means to estimate the unknown $\beta$.

We estimate $\beta$ using the estimator $\hat{\beta}$ that minimizes the quantity
\begin{align*}
\|\mathbf{y} - \mathbf{\hat{y}}\|_2^2 \quad=\quad \sum_{i=1}^n(y_i-\hat y_i)^2
\quad=\quad \sum_{i=1}^n(y_i-x_i^\top\hat\beta)^2
\quad=\quad (\mathbf{y}-\mathbf{X}\hat\beta)^\top(\mathbf{y}-\mathbf{X}\hat\beta)
\end{align*}
where $\hat{y}_i=x_i^\top\hat{\beta}$ is our *fitted value*. We will give a name to this quantity later.

The $\hat{\beta}$ that has this property is called the *least squares estimate* (LSE).

Assuming $\mathbf{X}$ has full column rank, the LSE has a closed-form given by
$$
\hat\beta = (\mathbf{X}^\top\mathbf{X})^{-1}\mathbf{X}^\top\mathbf{y} \;.
$$

(The full column rank assumption is so that $\mathbf{X}^\top\mathbf{X}$ is invertible.)

In [ ]:
df$enroll.num = match(df$Enrollment.time, c("2019 Winter", "2021 Fall", "2021 Winter", "2022 Fall", "2022 Winter",
                                            "2023 Summer")) - 1


In [ ]:
y = df$Overall.grade
x1 = df$Weekly.study.hours
x2 = df$Class.absences
x3 = df$enroll.num
x4 = df$Enjoyment


In [ ]:
# Create the design matrix (convert sex to binary with 'Female' as baseline)
X = cbind(1, x1, x2, x3, x4)
print(head(X))

       x1 x2 x3 x4
[1,] 1  5  1  1  5
[2,] 1 17  0  0  5
[3,] 1  3  0  5  5
[4,] 1  7  1  3  3
[5,] 1  7  0  3  3
[6,] 1  8  8  5  5


In [ ]:
print(head(y))

[1]  93  95 100  95  91  85


In [ ]:
# Compute the LSE manually
invXTX = solve(t(X) %*% X)
XTy = t(X) %*% y
beta_manual = invXTX %*% XTy
print(beta_manual) # Round to 3 decimals

         [,1]
   71.5770341
x1  0.2337244
x2 -0.5985353
x3  0.8197371
x4  3.9592104


In [ ]:
# Compare results with one-line R function for linear regression
# Don't use this on the assignments unless otherwise stated!
fit = lm(Overall.grade~ Weekly.study.hours+Class.absences+enroll.num+Enjoyment, data=df)

# Extract the coefficients from the fitted model
beta = coef(fit)
print(beta)

       (Intercept) Weekly.study.hours     Class.absences         enroll.num 
        71.5770341          0.2337244         -0.5985353          0.8197371 
         Enjoyment 
         3.9592104 


In [ ]:
summary(fit)


Call:
lm(formula = Overall.grade ~ Weekly.study.hours + Class.absences + 
    enroll.num + Enjoyment, data = df)

Residuals:
     Min       1Q   Median       3Q      Max 
-13.1473  -2.3486   0.2539   3.6471   8.9833 

Coefficients:
                   Estimate Std. Error t value Pr(>|t|)    
(Intercept)         71.5770     6.8853  10.396  1.6e-08 ***
Weekly.study.hours   0.2337     0.5148   0.454  0.65591    
Class.absences      -0.5985     0.5252  -1.140  0.27122    
enroll.num           0.8197     1.0567   0.776  0.44920    
Enjoyment            3.9592     1.2185   3.249  0.00503 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 6.343 on 16 degrees of freedom
Multiple R-squared:  0.6284,	Adjusted R-squared:  0.5355 
F-statistic: 6.763 on 4 and 16 DF,  p-value: 0.002193


In [ ]:
# Extract fitted values
yhat = fitted(fit)
print(head(yhat))

       1        2        3        4        5        6 
92.76291 95.34640 96.17294 86.95141 87.54995 92.55328 
